In [ ]:
folder_path = r"D:\Datasets\FLIM\ptu_files\z_stack_time_lapse\embryo_43pl_2ch_3tps"
folder_path = r"D:\Datasets\FLIM\ptu_files\z_stack\stack_smaller"
# folder_path = r"D:\Datasets\FLIM\sdt_files\Sample_files from Conni\sperm_seminal_receptacle_NADH_FLIM_2ch.sdt"
# folder_path = r"D:\Datasets\FLIM\ptu_files\single slice\raw_FLIM_data\single_FLIM_image.ptu"
# xml_path = None
x_pixel_size = 0.5*1e-6 # in m
y_pixel_size = 0.5*1e-6 # in m
z_pixel_size = 0.5*1e-6 # in m
pixel_size_unit = 'm'
time_resolution_per_slice = 0.663 # in seconds
time_unit = 's'
micro_time_resolution = 12.5 # in ns
micro_time_unit = 'ns'
channel_names = ['Centrosomes', 'Membrane and Chromatin', 'Bla']

In [4]:
from magicgui.tqdm import tqdm
from numcodecs import Blosc
import dask.array as da
import numpy as np
from pathlib import Path
from natsort import natsorted
from napari_flim_phasor_plotter._reader import get_read_function_from_extension, get_most_frequent_file_extension
from napari_flim_phasor_plotter._reader import get_max_slice_shape_and_dtype, get_structured_list_of_paths
from napari_flim_phasor_plotter._reader import get_max_zslices, get_max_time_points, ALLOWED_FILE_EXTENSION
from utilities import format_metadata, format_metadata2
import tifffile

folder_path = Path(folder_path)
file_extension = get_most_frequent_file_extension(folder_path)
if file_extension not in ALLOWED_FILE_EXTENSION:
    if file_extension == '':
        message = 'Please select a folder containing FLIM images.'
        print(message)
    else:
        message = 'Plugin does not support ' + \
            file_extension + ' . Supported file extensions are: '
        message += ', '.join(ALLOWED_FILE_EXTENSION[:-1])
        print(message)

# Get appropriate read function from file extension
imread = get_read_function_from_extension[file_extension]
if folder_path.is_file():
    print('Single file')
    data, flim_metadata = imread(folder_path)
    # Add channel dimension in case it is missing
    if len(data.shape) == 3:
        data = data[np.newaxis, :]
    # Add unitary axis for z
    data = data[:, :, np.newaxis]
    metadata_timelapse, metadata_single_timepoint = format_metadata2(
        flim_metadata=flim_metadata,
        stack_shape=data.shape,
        x_pixel_size=x_pixel_size,
        y_pixel_size=y_pixel_size,
        z_pixel_size=z_pixel_size,
        pixel_size_unit=pixel_size_unit,
        time_resolution_per_slice=time_resolution_per_slice,
        time_unit=time_unit,
        channel_names=channel_names,
        micro_time_resolution=micro_time_resolution,
        micro_time_unit=micro_time_unit,
        timelapse=False,)
    print(f"Saving OME-TIF...")
    output_path = folder_path.parent / 'OME-TIFs'
    output_path.mkdir(exist_ok=True)
    output_file_name = folder_path.stem + '.ome.tif'
    with tifffile.TiffWriter(output_path / output_file_name, ome=True) as tif:
        tif.write(data, metadata=metadata_single_timepoint, compression='zlib')
    print('Done')
else:
    print('Stack')

    # Get all file path with specified file extension
    file_paths = natsorted([file_path for file_path in folder_path.iterdir(
    ) if file_path.suffix == file_extension])
    # Get maximum shape and dtype from file names (file names must be in the format: "name_t000_z000")
    image_slice_shape, image_dtype = get_max_slice_shape_and_dtype(
        file_paths, file_extension)
    # If single channel, add a new axis
    if len(image_slice_shape) == 3:
        image_slice_shape = (1, *image_slice_shape)
    # Get maximum time and z from file names
    max_z = get_max_zslices(file_paths, file_extension)
    max_time_point = get_max_time_points(file_paths, file_extension)
    # Build stack shape with the fllowing convention: (channel, time, z, y, x)
    stack_shape = (
        image_slice_shape[0], max_time_point+1, max_z+1, *image_slice_shape[-2:])
    # Create an empty numpy array with the maximum shape and dtype
    numpy_array_summed_intensity = np.zeros(stack_shape, dtype=image_dtype)
    # Get a nested list of time point containing a list of z slices
    list_of_time_point_paths = get_structured_list_of_paths(
        file_paths, file_extension)

    output_path = folder_path / 'OME-TIFs'
    output_path.mkdir(exist_ok=True)
    timelapse = False
    if len(list_of_time_point_paths) > 1:
        timelapse = True

    for z_paths, t in zip(tqdm(list_of_time_point_paths, label='time_points'), range(len(list_of_time_point_paths))):
        stack_shape = (*image_slice_shape[:-2], max_z+1, *image_slice_shape[-2:]) # (channel, ut, z, y, x)
        numpy_array = np.zeros(stack_shape, dtype=image_dtype)
        for path, j in zip(tqdm(z_paths, label='z-slices'), range(len(z_paths))):
            data, flim_metadata = imread(path) # Read single file
            # Populate nuumpy array with data
            if len(data.shape) == 3:
                # Consider slice is single channel
                numpy_array[0, :data.shape[0],
                        j, :data.shape[1], :data.shape[2]] = data
            else:
                numpy_array[:data.shape[0], :data.shape[1],
                        j, :data.shape[2], :data.shape[3]] = data
        print(f"Processing timepoint: {t}")
        numpy_array_summed_intensity[:, t, :, :, :] = np.sum(numpy_array, axis=1, dtype=image_dtype) # channel, z, y, x
        # Extract metadata
        metadata_timelapse, metadata_single_timepoint = format_metadata2(
            flim_metadata=flim_metadata,
            stack_shape=numpy_array.shape,
            x_pixel_size=x_pixel_size,
            y_pixel_size=y_pixel_size,
            z_pixel_size=z_pixel_size,
            pixel_size_unit=pixel_size_unit,
            time_resolution_per_slice=time_resolution_per_slice,
            time_unit=time_unit,
            channel_names=channel_names,
            micro_time_resolution=micro_time_resolution,
            micro_time_unit=micro_time_unit,
            timelapse=timelapse,)
        print(f"Saving OME-TIF...")

        if timelapse:
            t_string = str(t).zfill(len(str(len(list_of_time_point_paths))))
            output_file_name =  folder_path.stem + f'_t{t_string}.ome.tif'
        else:
            output_file_name = folder_path.stem + '.ome.tif'
        with tifffile.TiffWriter(output_path / output_file_name, ome=True) as tif:
            tif.write(numpy_array, metadata=metadata_single_timepoint, compression='zlib')
    output_file_name = folder_path.stem + '_summed_intensity.ome.tif'
    with tifffile.TiffWriter(output_path / output_file_name, ome=True) as tif:
        tif.write(numpy_array_summed_intensity[:], metadata=metadata_timelapse, compression='zlib')

    print('Done')




# # zarr file will be saved in the same folder as the input folder
# output_path = folder_path / (folder_path.stem + '.zarr')
# # Using zarr to automatically guess chunk sizes
# # Use Blosc compressor for better compression and speed
# compressor = Blosc(cname='zstd', clevel=3, shuffle=Blosc.BITSHUFFLE)
# # Create an empty zarr array of a specified shape and dtype filled with zeros
# zarr_array = zarr.open(output_path, mode='w',
#                         shape=stack_shape, dtype=image_dtype, compressor=compressor)
# # Using dask to rechunk micro-time axis in single chunk (for fft calculation afterwards)
# dask_array = da.from_zarr(output_path)
# # Rechunk axis 1 (micro-time axis) to a single chunk
# dask_array = dask_array.rechunk(chunks={1: -1})
# # Overwriting previous zarr rechunked
# da.to_zarr(dask_array, output_path, overwrite=True)
# # Read zarr as read/write
# zarr_array = zarr.open(output_path, mode='r+')
# # Fill zarr array with data
# for z_paths, i in zip(tqdm(list_of_time_point_paths, label='time_points'), range(len(list_of_time_point_paths))):
#     for path, j in zip(tqdm(z_paths, label='z-slices'), range(len(z_paths))):
#         data, metadata_list = imread(path)
#         # If single channel, add a new axis
#         if len(data.shape) == 3:
#             zarr_array[0, :data.shape[0], i,
#                     j, :data.shape[1], :data.shape[2]] = data
#         else:
#             zarr_array[:data.shape[0], :data.shape[1], i,
#                     j, :data.shape[2], :data.shape[3]] = data
# zarr_metadata = dict()
# for i, metadata in enumerate(metadata_list):
#     zarr_metadata['channel ' + str(i)] = metadata
#     zarr_array.attrs.update(zarr_metadata)
# print('Done')

Stack


100%|██████████| 43/43 [00:08<00:00,  5.07it/s]


Processing timepoint: 0
Saving OME-TIF...


100%|██████████| 1/1 [00:11<00:00, 11.02s/it]

Done


In [ ]:
# Additional metadata in case the xml file is not available
xml_path = None
z_pixel_size = 0.5*1e-6 # in m
pixel_size_unit = 'm'
time_resolution_per_slice = 0.663 # in seconds
time_unit = 's'
channel_names = ['Centrosomes', 'Membrane and Chromatin']

In [ ]:
# Read the zarr file
        data, flim_metadata = read_stack(zarr_data_path)
        summed_intensity_stack = da.sum(data, axis=1).astype(np.uint32)
        print("Extracting metadata...")
        if xml_path is None:
            print("No xml file found. Using additional metadata.")
        else:
            if not xml_path.exists():
                print("The xml file does not exist. Using additional metadata.")
            else:
                print(f"Reading metadata from: {xml_path} and zarr file.")
        # Extract metadata
        metadata_timelapse, metadata_single_timepoint = format_metadata(
            flim_metadata,
            xml_path, 
            data.shape,
            z_pixel_size,
            pixel_size_unit,
            time_resolution_per_slice,
            time_unit,
            channel_names)

        laser_frequency = flim_metadata[0]['frequency']/1e6 # in MHz
        print(f'Laser frequency: {laser_frequency} MHz')
        # Split channels
        image_raw_FLIM_channel_0 = data[0] # first channel
        image_raw_FLIM_channel_1 = data[1] # second channel

Save summed intensity stack (collapse flim dimension)

In [ ]:
output_file_name = sample_name + '_summed_intensity_with_segmentation.ome.tif'
        with tifffile.TiffWriter(omero_output_path / output_file_name, ome=True) as tif:
            tif.write(summed_intensity_stack[:], metadata=metadata_timelapse, compression='zlib')

Save each datapoint as individual tif

In [ ]:
for t in range(image_raw_FLIM_channel_0.shape[1]):
            print(f"Processing timepoint: {t}")
            print(f"Saving OME-TIF...")
            t_string = str(t).zfill(len(str(data.shape[2])))
            output_file_name =  sample_name + f'_t{t_string}.ome.tif'
            with tifffile.TiffWriter(omero_output_path / output_file_name, ome=True) as tif:
                tif.write(data[:,:,t], metadata=metadata_single_timepoint, compression='zlib')